The lattice Wess-Zumino Hamiltonian is given by 

$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

For conceptual and code clarity we break this into four terms to specify contributions for interactions(I), non-interactions(NI) of terms with purely bosonic(B) or fermionic(F)/mixed terms.

$$
  H^B_{NI}=\frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2 \\ 
  H^B_{I}=\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \\
  H^F_{NI}=\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \\ 
  H^F_{I}=(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
$$

In [186]:
import scipy
from scipy.sparse import eye, kron, coo_matrix
from scipy.sparse.linalg import eigsh

import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [218]:
# Parameters
N = 2
cutoff = 16
a = 1.0

In [219]:
def kron_tensor(size, site, total_sites, operator):

    I = eye(size, format='coo', dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
  
    return reduce(kron, operators) 

In [220]:
q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

q_0 = kron_tensor(cutoff, 0, N, q)
q_1 = kron_tensor(cutoff, 1, N, q)
p_0 = kron_tensor(cutoff, 0, N, p)
p_1 = kron_tensor(cutoff, 1, N, p)
chi_0 = kron_tensor(2, 0, N, chi)
chi_1 = kron_tensor(2, 1, N, chi)
chidag_0 = kron_tensor(2, 0, N, chidag)
chidag_1 = kron_tensor(2, 1, N, chidag)

In [221]:
boson = 0.5*(p_0 @ p_0) + 0.5*(p_1 @ p_1) + 0.625*(q_0 @ q_0) + 0.625*(q_1 @ q_1)

In [222]:
fermion = (chidag_0 @ chi_0) - (chidag_1 @ chi_1) + 0.5*(chidag_0 @ chi_1) + 0.5*(chidag_1 @ chi_0)

In [223]:
print(boson)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 1152 stored elements and shape (256, 256)>
  Coords	Values
  (0, 2)	(0.08838834764831843+0j)
  (0, 0)	(1.1249999999999998+0j)
  (0, 32)	(0.08838834764831843+0j)
  (1, 3)	(0.15309310892394867+0j)
  (1, 1)	(2.25+0j)
  (1, 33)	(0.08838834764831843+0j)
  (2, 0)	(0.08838834764831843+0j)
  (2, 4)	(0.2165063509461097+0j)
  (2, 2)	(3.375+0j)
  (2, 34)	(0.08838834764831843+0j)
  (3, 1)	(0.15309310892394867+0j)
  (3, 5)	(0.2795084971874737+0j)
  (3, 3)	(4.499999999999999+0j)
  (3, 35)	(0.08838834764831843+0j)
  (4, 2)	(0.2165063509461097+0j)
  (4, 6)	(0.34232659844072866+0j)
  (4, 4)	(5.624999999999999+0j)
  (4, 36)	(0.08838834764831843+0j)
  (5, 3)	(0.2795084971874737+0j)
  (5, 7)	(0.40504629365049105+0j)
  (5, 5)	(6.749999999999998+0j)
  (5, 37)	(0.08838834764831843+0j)
  (6, 4)	(0.34232659844072866+0j)
  (6, 8)	(0.46770717334674283+0j)
  (6, 6)	(7.874999999999998+0j)
  :	:
  (249, 251)	(0.6555055301063444+0j)
  (249, 217)	(0.905

In [224]:
print(fermion)

<Block Sparse Row sparse matrix of dtype 'complex128'
	with 16 stored elements (blocksize=2x2) and shape (4, 4)>
  Coords	Values
  (0, 0)	0j
  (0, 1)	0j
  (1, 0)	0j
  (1, 1)	(-1+0j)
  (0, 2)	0j
  (0, 3)	0j
  (1, 2)	(0.5+0j)
  (1, 3)	0j
  (2, 0)	0j
  (2, 1)	(0.5+0j)
  (3, 0)	0j
  (3, 1)	0j
  (2, 2)	(1+0j)
  (2, 3)	0j
  (3, 2)	0j
  (3, 3)	0j


In [225]:
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')

In [226]:
boson_kron = kron(boson, I_f, format='coo')
fermion_kron = kron(I_b, fermion, format='coo')

In [227]:
H = boson_kron + fermion_kron

In [228]:
fermion_kron

<COOrdinate sparse matrix of dtype 'complex128'
	with 4096 stored elements and shape (1024, 1024)>

In [229]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 4
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [3.33479721e-16 1.11803399e+00 1.11803399e+00 1.11803399e+00]


In [239]:
# Parameters
N = 2
cutoff = 16
a = 1.0

I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')

b_dim = (I_b.size,I_b.size)
f_dim = (I_f.size,I_f.size)
dim = I_b.size * I_f.size

zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

H_b = coo_matrix(b_dim, dtype=np.complex128)
H_f = coo_matrix(f_dim, dtype=np.complex128)
H_bi = coo_matrix((dim, dim), dtype=np.complex128)

H = coo_matrix((dim, dim), dtype=np.complex128)

q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

for n in range(N):

    print(n)
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = (a / 2) * coo_matrix(W_prime @ W_prime)

    if n == 0:
        q_nm1 = zero_qop
        q_np1 = kron_tensor(cutoff, (n + 1), N, q)
        chi_np1 = kron_tensor(2, (n + 1), N, chi)
        chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
    else:
        q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
        q_np1 = zero_qop
        chidag_np1 = zero_cop
        chi_np1 = zero_cop

    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = (a / 2) * (gradient @ gradient)

    potential_gradient_term = a * (W_prime @ gradient)

    H_b += (p2 + potential + gradient_term + potential_gradient_term)

    # Fermionic term
    #commutator_term = kron(W_double_prime, ((-1) ** n) * coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')
    commutator_term = kron(W_double_prime, ((-1) ** n) * coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')
    H_bi += commutator_term

    fermion_hopping = 0.5*(chidag_n @ chi_np1 + chidag_np1 @ chi_n)
    H_f += (fermion_hopping) #+ commutator_term

H = kron(H_b, I_f, format='coo') + kron(I_b, H_f, format='coo') + H_bi
   

0
1


In [240]:
print(H_b)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 1152 stored elements and shape (256, 256)>
  Coords	Values
  (0, 32)	(0.08838834764831843+0j)
  (0, 0)	(1.1249999999999998+0j)
  (0, 2)	(0.08838834764831843+0j)
  (1, 33)	(0.08838834764831843+0j)
  (1, 1)	(2.25+0j)
  (1, 3)	(0.15309310892394862+0j)
  (2, 34)	(0.08838834764831843+0j)
  (2, 2)	(3.375+0j)
  (2, 4)	(0.21650635094610962+0j)
  (2, 0)	(0.08838834764831843+0j)
  (3, 35)	(0.08838834764831843+0j)
  (3, 3)	(4.499999999999999+0j)
  (3, 5)	(0.27950849718747367+0j)
  (3, 1)	(0.15309310892394862+0j)
  (4, 36)	(0.08838834764831843+0j)
  (4, 4)	(5.624999999999999+0j)
  (4, 6)	(0.3423265984407287+0j)
  (4, 2)	(0.21650635094610962+0j)
  (5, 37)	(0.08838834764831843+0j)
  (5, 5)	(6.749999999999998+0j)
  (5, 7)	(0.40504629365049116+0j)
  (5, 3)	(0.27950849718747367+0j)
  (6, 38)	(0.08838834764831843+0j)
  (6, 6)	(7.874999999999998+0j)
  (6, 8)	(0.46770717334674267+0j)
  :	:
  (249, 249)	(19.125+0j)
  (249, 251)	(0.65550553010

In [241]:
print(H_f)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 2 stored elements and shape (4, 4)>
  Coords	Values
  (1, 2)	(0.5+0j)
  (2, 1)	(0.5+0j)


In [243]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 6
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [-1.34584110e-15  1.11803399e+00  1.11803399e+00  1.11803399e+00
  1.11803399e+00  2.23606798e+00]


In [ ]:
# Parameters
N = 2
cutoff = 16
a = 1.0

I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')

b_dim = (I_b.size,I_b.size)
f_dim = (I_f.size,I_f.size)
dim = I_b.size * I_f.size

zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

H_b = coo_matrix(b_dim, dtype=np.complex128)
H_f = coo_matrix(f_dim, dtype=np.complex128)
H_bi = coo_matrix((dim, dim), dtype=np.complex128)

H = coo_matrix((dim, dim), dtype=np.complex128)

q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

for n in range(N):

    print(n)
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = (a / 2) * coo_matrix(W_prime @ W_prime)

    if n == 0:
        q_nm1 = zero_qop
        q_np1 = kron_tensor(cutoff, (n + 1), N, q)
        chi_np1 = kron_tensor(2, (n + 1), N, chi)
        chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
    else:
        q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
        q_np1 = zero_qop
        chidag_np1 = zero_cop
        chi_np1 = zero_cop

    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = (a / 2) * (gradient @ gradient)

    potential_gradient_term = a * (W_prime @ gradient)

    H_b += (p2 + potential + gradient_term + potential_gradient_term)

    # Fermionic term
    #commutator_term = kron(W_double_prime, ((-1) ** n) * coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')
    commutator_term = ((-1) ** n) * coo_matrix(chidag_n @ chi_n - 0.5 * I_f)
    #H_bi += commutator_term

    fermion_hopping = 0.5*(chidag_n @ chi_np1 + chidag_np1 @ chi_n)
    H_f += (fermion_hopping + commutator_term)

H = kron(H_b, I_f, format='coo') + kron(H_f, I_b)
   

dirichlet
0
dirichlet
1 Zero
